In [70]:

import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [71]:

headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [72]:
product_url=input()
# https://www.amazon.com/Invicta-8926OB-Stainless-Automatic-Bracelet/dp/B000JQFX1G/?_encoding=UTF8&pd_rd_w=oivdj&content-id=amzn1.sym.6cf7c564-ca00-466b-86d0-47197a966847&pf_rd_p=6cf7c564-ca00-466b-86d0-47197a966847&pf_rd_r=B6N0XX1NXHP4ZV6JZ6RQ&pd_rd_wg=4JYln&pd_rd_r=7c96c10f-2150-47d9-bdc2-27cabcad19ba&ref_=pd_hp_d_btf_nta-top-sellers&th=1



 https://www.amazon.in/BLUETYGA-SUNSCREEN-Workout-Trekking-Sunscreen/dp/B0CB8BR154/ref=sr_1_1?_encoding=UTF8&content-id=amzn1.sym.74f25a9d-e850-443b-a26a-da459bed7e95&dib=eyJ2IjoiMSJ9.zsTl9IMo6YLaoaZdKU5DzIdH712e5Z_dfqNpuiqDvNT2JEClhCiHUczKQ_uORUXuj5VMOhP14NwTDLLnfKASqrYA-SlxPsd5btlbc-nhCPpLnMRoL0kTzXzHFlClPSssa3dboBiTJdyMjZGw3WIxEjPZyY7vC8FfHpcTZ5D2Q4nhcpyhpNOoEAgwuNoicNfchD-arKXAiPukTpNQvBbUusAenCqVOMHJP9-aUnjx1B6rSFW6ELJSIV4GEXywYoAJ0dwYlbPGqRvdNECwE6mX-YAt30PMXDhXQxL5vuzP4_g.8QRmSLZBs3ymiWYSBCngQX9cj3OffP3j0aRpki18T5s&dib_tag=se&pd_rd_r=61d14ad9-ed38-46a5-8837-e80d4569abff&pd_rd_w=kN9iI&pd_rd_wg=5ws4x&pf_rd_p=74f25a9d-e850-443b-a26a-da459bed7e95&pf_rd_r=2T9QDJVDG5HYWT2WZFQW&qid=1726738719&refinements=p_n_specials_match%3A21618256031%2Cp_85%3A10440599031%2Cp_n_feature_nineteen_browse-bin%3A11301357031&rnid=11301356031&rps=1&s=apparel&sr=1-1


In [75]:
import re


def extract_product_info(url):
    comf=False
   
    pattern = r"amazon\.in/([^/]+)/dp/([^/?]+)" 

        
    
   
    match = re.search(pattern, url)
    if not match:
        pattern=r"amazon\.com/([^/]+)/dp/([^/?]+)" 
        match = re.search(pattern, url)
        comf=True
        
    
    if match:
        product_name = match.group(1)
        product_id = match.group(2)
        return product_name, product_id,comf
    else:
        return None, None,True





In [76]:

product_name, product_id,comf = extract_product_info(product_url)

if product_name and product_id:
    print(f"Product Name: {product_name}")
    print(f"Product ID: {product_id}")
else:
    print("Product information not found.")

Product Name: BLUETYGA-SUNSCREEN-Workout-Trekking-Sunscreen
Product ID: B0CB8BR154


In [102]:

reviews_url = 'https://www.amazon.com/'+product_name+'/product-reviews/'+product_id

    

In [103]:
reviews_url

'https://www.amazon.com/BLUETYGA-SUNSCREEN-Workout-Trekking-Sunscreen/product-reviews/B0CB8BR154'

In [104]:

len_page = 4

In [105]:

def reviewsHtml(url, len_page):
    
    
    soups = []
    
    
    for page_no in range(1, len_page + 1):
        
        
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        
        response = requests.get(url, headers=headers)
        
      
        soup = BeautifulSoup(response.text, 'lxml')
        
        
        soups.append(soup)
        
    return soups

In [106]:

def getReviews(html_data):

    data_dicts = []
    
    
    boxes = html_data.select('div[data-hook="review"]')
    
     
    for box in boxes:
        
      
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
           
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        
        data_dicts.append(data_dict)
    
    return data_dicts

In [107]:

html_datas = reviewsHtml(reviews_url, len_page)

In [108]:

reviews = []

In [109]:

for html_data in html_datas:
    
   
    review = getReviews(html_data)
    
  
    reviews += review

In [110]:

df_reviews = pd.DataFrame(reviews)
df_reviews

""


In [20]:

df_reviews.to_csv('reviews.csv', index=False)